<a href="https://colab.research.google.com/github/amanglarus/image_classification_using-tensorflow/blob/main/classification_using_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/tshirt_laptop_data.zip /content/

In [ ]:
!unzip tshirt_laptop_data.zip

# Install specific tensorflow version 

In [ ]:
!pip install tensorflow-gpu==2.4.0

In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.0


# Load data in pipe line to tarin the model

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense,Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model, Sequential

In [3]:
NUM_CLASSES = 2
BATCH_SIZE=32
IMAGE_SIZE = 48
NUM_CHANNEL = 3

IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
MODEL_IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNEL)

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir='/content/tshirt_laptop_data/train'
val_data_dir='/content/tshirt_laptop_data/test'

# from tensorflow.keras.layers.preprocessing.image_preprocessing import HORIZONTAL

train_datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=10,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 horizontal_flip=True,
                                 fill_mode='nearest')

val_datagen=ImageDataGenerator(rescale=1./255)




train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=IMAGE_SHAPE,
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='sparse')

val_generator=val_datagen.flow_from_directory(val_data_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode='sparse')






Found 2385 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


# Build Model

### Mobilenet V3 from scratch


In [46]:
import tensorflow as tf

def conv_block(inputs, filters, kernel_size, strides, use_bias=True):
    x = tf.keras.layers.Conv2D(filters, kernel_size, strides=strides, padding='same', use_bias=use_bias)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    return x

def bottleneck(inputs, filters, kernel_size, expansion_factor, strides, use_bias=True):
    in_channels = inputs.shape[-1]
    x = conv_block(inputs, expansion_factor * in_channels, 1, 1, use_bias)
    x = conv_block(x, expansion_factor * in_channels, kernel_size, strides, use_bias)
    x = tf.keras.layers.Conv2D(filters, 1, strides=1, padding='same', use_bias=use_bias)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if strides == 1 and in_channels == filters:
        x = tf.keras.layers.Add()([x, inputs])
    return x

def mobilenetv3_small(input_shape, num_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = conv_block(inputs, 8, 3, strides=2)
    x = bottleneck(x, 8, 3, expansion_factor=1, strides=2)
    x = bottleneck(x, 8, 3, expansion_factor=1, strides=1)
    x = bottleneck(x, 8, 3, expansion_factor=1, strides=1)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=2)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=1)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=1)
    x = conv_block(x, 8, 1, strides=1)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(48)(x)
    x = tf.keras.layers.ReLU(6.)(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs, x)

# example usage
model = mobilenetv3_small(input_shape=MODEL_IMAGE_SHAPE, num_classes=NUM_CLASSES)
model.summary()


Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 24, 24, 8)    224         input_16[0][0]                   
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 24, 24, 8)    32          conv2d_95[0][0]                  
__________________________________________________________________________________________________
re_lu_72 (ReLU)                 (None, 24, 24, 8)    0           batch_normalization_95[0][0]     
____________________________________________________________________________________________

### Mobilenet V2 from scratch

In [ ]:
import tensorflow as tf

def conv_block(inputs, filters, kernel_size, strides):
    x = tf.keras.layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    return x

def bottleneck(inputs, filters, kernel_size, t, strides, use_bias=False):
    in_channels = inputs.shape[-1]
    x = conv_block(inputs, t * in_channels, 1, 1)
    x = tf.keras.layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same', use_bias=use_bias)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv2D(filters, 1, strides=1, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if strides == 1 and in_channels == filters:
        x = tf.keras.layers.Add()([x, inputs])
    return x

def mobilenetv2(input_shape, num_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = conv_block(inputs, 32, 3, strides=2)
    x = bottleneck(x, 16, 3, t=1, strides=1)
    x = bottleneck(x, 24, 3, t=6, strides=2)
    x = bottleneck(x, 24, 3, t=6, strides=1)
    x = bottleneck(x, 32, 3, t=6, strides=2)
    x = bottleneck(x, 32, 3, t=6, strides=1)
    x = bottleneck(x, 64, 3, t=6, strides=2)
    x = bottleneck(x, 64, 3, t=6, strides=1)
    x = bottleneck(x, 96, 3, t=6, strides=1)
    x = bottleneck(x, 96, 3, t=6, strides=1)
    x = bottleneck(x, 160, 3, t=6, strides=2)
    x = bottleneck(x, 160, 3, t=6, strides=1)
    x = bottleneck(x, 320, 3, t=6, strides=1)
    x = conv_block(x, 1280, 1, strides=1)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs, x)

# example usage
model = mobilenetv2(input_shape=(224, 224, 3), num_classes=10)
model.summary()


In [21]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications import MobileNetV3Large
#tf.keras.applications.MobileNetV3Small

base_model = MobileNetV3Small(
    input_shape=MODEL_IMAGE_SHAPE,
    alpha=0.2,
    include_top=False,
    weights=None, #  'imagenet'
    #input_tensor=None,
    #pooling=None,
    #classes=1000,
    #classifier_activation='softmax',
    #**kwargs
)

for layer in base_model.layers: #base_model.layers[:-6] means that consider layer only till 6th last layer from layer 1
    layer.trainable = True  



# penultimate_layer = model.layers[-2]
# new_top_layer = tf.keras.layers.Dense(1)(penultimate_layer.output)  # create new FC layer and connect it to the rest of the model
# new_model = tf.keras.models.Model(model.input, new_top_layer)



# Use a Sequential model to add a trainable classifier on top
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax"),
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
MobilenetV3small (Functional (None, 2, 2, 1024)        231608    
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1024)              0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                65600     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 130       
Total params: 297,338
Trainable params: 294,058
Non-trainable params: 3,280
____________________________________________

# Train the model

In [47]:
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("image_classification_checkpoint.h5",
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=2,
                          verbose=1,
                          restore_best_weights=True)

callbacks=[checkpoint,earlystop]

model.compile(loss='SparseCategoricalCrossentropy',
                   optimizer=Adam(learning_rate=0.0015),
                   metrics=['sparse_categorical_accuracy'])

epochs=2

batch_size = 32

history = model.fit(train_generator,
                        #  steps_per_epoch=num_train_samples//batch_size,
                         epochs=epochs,
                        #  callbacks=callbacks,
                         validation_data=val_generator,
                        #  validation_steps=num_val_samples//batch_size
                         )

Epoch 1/2
75/75 [==============================] - 20s 248ms/step - loss: 0.6665 - sparse_categorical_accuracy: 0.6071 - val_loss: 0.7084 - val_sparse_categorical_accuracy: 0.5150
Epoch 2/2
75/75 [==============================] - 18s 242ms/step - loss: 0.5991 - sparse_categorical_accuracy: 0.6756 - val_loss: 0.8340 - val_sparse_categorical_accuracy: 0.5000


# Convert from tensorflow to tensorflow lite

In [25]:
datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=10,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 horizontal_flip=True,
                                 fill_mode='nearest')
generator=datagen.flow_from_directory(val_data_dir,
                                      target_size=IMAGE_SHAPE,
                                      batch_size=BATCH_SIZE,
                                      class_mode='sparse')
def representative_data_gen():
  i = 0
  for image_batch, labels_batch in generator:
    i = i+1
    if i > 20:
      break;
    yield [image_batch]

Found 200 images belonging to 2 classes.


In [48]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

print('\nSetting the optimization flags..')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

print('\nConverting..')
tflite_model = converter.convert()

open("person_detect_model_data.tflite", "wb").write(tflite_model)

print("Done Conversion")


Setting the optimization flags..

Converting..
Done Conversion


# Convert from tensorflow to c array

In [49]:
!xxd -i ./person_detect_model_data.tflite > person_detect_model_data.cpp

In [39]:
#!cp /content/person_detect_model_data.cpp /content/person_detect_model_data_copy.cpp

In [50]:

search_text1 = "unsigned char __person_detect_model_data_tflite[]"


replace_text1 = str('#include "person_detect_model_data.h" \n\
#ifdef __has_attribute \n\
#define HAVE_ATTRIBUTE(x) __has_attribute(x) \n\
#else \n\
#define HAVE_ATTRIBUTE(x) 0 \n\
#endif \n\
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__)) \n\
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4))) \n\
#else \n\
#define DATA_ALIGN_ATTRIBUTE \n\
#endif \n\
const unsigned char g_person_detect_model_data[] DATA_ALIGN_ATTRIBUTE')


search_text2 = "unsigned int __person_detect_model_data_tflite_len"

replace_text2 = "const int g_person_detect_model_data_len"

with open(r'/content/person_detect_model_data.cpp', 'r') as file:

    data = file.read()

    data = data.replace(search_text1, replace_text1)
    data = data.replace(search_text2, replace_text2)
  # data = data.replace(search_text2, replace_text2)

with open(r'/content/person_detect_model_data.cpp', 'w') as file:
    file.write(data)
    file.close()



print("Text replaced")


Text replaced
